<a href="https://colab.research.google.com/github/sunilchinnahalli/ASS_9_9.1_9.2_9.3/blob/master/userlevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re

from keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate
from keras.models import Model
from keras.utils import plot_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
bot_accounts = pd.concat([pd.read_csv('/content/drive/My Drive/twitter/social_spambots_1.csv'), pd.read_csv('/content/drive/My Drive/twitter/social_spambots_2.csv'), pd.read_csv('/content/drive/My Drive/twitter/social_spambots_3.csv')]).reset_index(drop=True)
clean_accounts = pd.read_csv('/content/drive/My Drive/twitter/geniune_accounts.csv')

requiredColumns = ['listed_count', 'geo_enabled', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'profile_use_background_image', 'protected', 'default_profile']
bot_accounts = bot_accounts[requiredColumns]
clean_accounts = clean_accounts[requiredColumns]
def clean_df(df):
    
    
    df['listed_count'] = df['listed_count'].apply(lambda x: x if x==x else 0)
    df['geo_enabled'] = df['geo_enabled'].apply(lambda x: 1 if x==x else 0)
    df['statuses_count'] = df['statuses_count'].apply(lambda x: x if x==x else 0)
    df['friends_count'] = df['friends_count'].apply(lambda x: x if x==x else 0)
    df['followers_count'] = df['followers_count'].apply(lambda x: x if x==x else 0)
    df['favourites_count'] = df['favourites_count'].apply(lambda x: x if x==x else 0)
    df['profile_use_background_image'] = df['profile_use_background_image'].apply(lambda x: 1 if x==x else 0)
    df['verified']=df['verified'].apply(lambda x: 1 if x==x else 0)
    df['is_protected']=df['protected'].apply(lambda x: 1 if x==x else 0)
    df['default_profile'] = df['default_profile'].apply(lambda x: 0 if x==x else 1)
    #df = df.rename(index=str, columns={"screen_name": "username", "statuses_count": "total_tweets", "friends_count": "total_following", "followers_count": "total_followers", "favourites_count": "total_likes"})
    return df[['listed_count', 'geo_enabled', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'profile_use_background_image', 'is_protected', 'default_profile']]

bot_accounts = clean_df(bot_accounts)
clean_accounts = clean_df(clean_accounts)
#print(bot_accounts)

In [0]:
bot_accounts['BotOrNot'] = 1
clean_accounts['BotOrNot'] = 0
#print(bot_accounts)
#print(clean_accounts)

combined_df = pd.concat([bot_accounts, clean_accounts])

new_df = combined_df.sample(frac=1).reset_index(drop=True)

In [6]:
training_df = new_df[:int(combined_df.shape[0] * 0.8)]
test_df = new_df[int(combined_df.shape[0] * 0.8):]



columns_to_standardize = ['listed_count', 'geo_enabled', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'profile_use_background_image', 'is_protected', 'default_profile']

training_df_mean = training_df[columns_to_standardize].mean()
training_df_std = training_df[columns_to_standardize].std()

training_df[columns_to_standardize] = (training_df[columns_to_standardize] - training_df_mean)/training_df_std
test_df[columns_to_standardize] = (test_df[columns_to_standardize] - training_df_mean)/training_df_std

# training_df_mean = training_df.mean()
# training_df_std = training_df.std()

# training_df = (training_df - training_df_mean)/training_df_std
# test_df = (test_df - training_df_mean)/training_df_std

# max_vals = training_df.max()

# training_df = training_df/max_vals
# test_df = test_df/max_vals

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [7]:
#X_train = training_df.drop(['BotOrNot', 'is_protected'], axis=1).values
#y_train = training_df['BotOrNot'].values.reshape(-1,1)
#print(X_train)
X_train = training_df
y_train = training_df['BotOrNot'].values.reshape(-1,1)

X_test = test_df
y_test = test_df['BotOrNot'].values.reshape(-1,1)
print()
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
sm = SMOTEENN()
X_train, y_train = sm.fit_sample(X_train, y_train)
#print(X_train)
from sklearn.ensemble import AdaBoostClassifier

# Import Support Vector Classifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
svc=SVC(probability=True, kernel='linear')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9958283671036948


In [0]:
#from sklearn.metrics import classification_report
#predicted_classes = model.predict(np.array(X_test))
#target_names = ["Class {}".format(i) for i in range(2)]


In [0]:
#predicted_classes = np.argmax(np.round(predicted_classes))
#print(classification_report(y_train, predicted_classes, target_names=target_names))
def get_accuracy(AL, y, verbose=1):
    
    try:
        AL = np.array(AL)
        y = np.array(y)

        AL = AL.reshape(-1)
        y = y.reshape(-1)

        AL = AL > 0.5
        AL = AL.astype(int)

        y = y > 0.5
        y = y.astype(int)

        total = AL.shape[0]

        TP = np.sum(np.logical_and(AL==1, y==1))
        TN = np.sum(np.logical_and(AL==0, y==0))

        FP = np.sum(np.logical_and(AL==1, y==0))
        FN = np.sum(np.logical_and(AL==0, y==1))

        P = TP / (TP + FP)
        R = TP / (TP + FN)
        F1 = (2 * P * R) / (P + R)


        acc = np.sum(AL == y)/total


        if verbose == 1:
            print("\nAccuracy: {} \n".format(acc))
            print("True Positive: {} \nTrue Negative: {}\nFalse Positive: {} \nFalse Negative: {}\n".format(TP, TN, FP, FN))
            print("Precision: {} \nRecall: {} \nF1 Score: {}\n".format(P, R, F1))
        
        return acc
    except:
        return 0

In [16]:
y_predict = abc.predict(X_train)
get_accuracy(y_predict, y_train)


Accuracy: 1.0 

True Positive: 3894 
True Negative: 3905
False Positive: 0 
False Negative: 0

Precision: 1.0 
Recall: 1.0 
F1 Score: 1.0



1.0

In [19]:
y_test_predict = abc.predict(X_test)
get_accuracy(y_test_predict, y_test)


Accuracy: 0.9958283671036948 

True Positive: 987 
True Negative: 684
False Positive: 0 
False Negative: 7

Precision: 1.0 
Recall: 0.9929577464788732 
F1 Score: 0.9964664310954063



0.9958283671036948